In [133]:
%load_ext autoreload
%autoreload 2
import dataloader
import tomotopy as tp
from itertools import chain
import tqdm
import pandas as pd
import numpy as np
import slda
import post_classifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
POSTPATH = './Data/crowd/train/shared_task_posts.csv'
LABELPATH = './Data/crowd/train/crowd_train.csv'
USERPATH = './Data/crowd/train/task_C_train.posts.csv'

users = dataloader.load_user_subset_from_train(USERPATH, subset = 1000)
    
user_to_post, post_to_words, post_to_metadata = dataloader.load_posts(POSTPATH, user_subset = users, append_title = True)
post_to_label = dataloader.load_classification(LABELPATH, user_to_post, post_to_words, post_to_metadata)
filtered_data, sw_posts, sw_timestamps = dataloader.filter_posts(post_to_label, post_to_metadata)
print(len(filtered_data))
filtered_data = dataloader.filter_near_SW(filtered_data,post_to_metadata, sw_timestamps)
print(len(filtered_data))

filtered_data = dataloader.filter_stopwords(filtered_data)
sw_posts = dataloader.filter_stopwords(sw_posts)


FOLDERPATH = './Processing/crowd_processed/'
dataloader.save_to_folder(FOLDERPATH, user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps)


  0%|          | 5813/2038753 [00:00<00:34, 58125.34it/s]


Filtering subset...


 45%|████▍     | 910635/2038753 [00:13<00:16, 69271.19it/s]


 87%|████████▋ | 1775524/2038753 [00:26<00:04, 64142.29it/s]


100%|██████████| 2038753/2038753 [00:30<00:00, 67373.25it/s]

  1%|          | 485/57015 [00:00<00:11, 4840.64it/s]

...


 97%|█████████▋| 55294/57015 [00:20<00:00, 4604.66it/s]


100%|██████████| 57015/57015 [00:20<00:00, 2731.27it/s]

  0%|          | 0/55255 [00:00<?, ?it/s]

55255
Filtering posts far away from SW posts...



100%|██████████| 55255/55255 [00:00<00:00, 165944.03it/s]


29200


In [30]:
FOLDERPATH = './Processing/crowd_processed/'
user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps = dataloader.load_from_folder(FOLDERPATH)

In [31]:
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'a']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'b']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'c']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'd']))
print(len(filtered_data))

878
203
632
2025
29200


In [143]:
model = slda.train_slda_model_from_data(filtered_data, topics=40)

100%|██████████| 29200/29200 [00:00<00:00, 78822.29it/s]


Beginning sLDA training...
Iteration: 0	Log-likelihood: -10.110838094061865
Iteration: 100	Log-likelihood: -9.192389218889685
Iteration: 200	Log-likelihood: -9.085921304964836
Iteration: 300	Log-likelihood: -9.041743725768162
Iteration: 400	Log-likelihood: -9.020902081291336
Iteration: 500	Log-likelihood: -9.003523706251233
Iteration: 600	Log-likelihood: -8.989685635618633
Iteration: 700	Log-likelihood: -8.980501838181022
Iteration: 800	Log-likelihood: -8.963619686414482
Iteration: 900	Log-likelihood: -8.956399065113708
Finished Training


In [144]:
slda_coefficients = model.get_regression_coef(0)
data = []
for k in range(model.k):
    top_words = model.get_topic_words(k, top_n=40)
    words = [word for (word, float) in top_words]
    words = ", ".join(words)
    data.append([words, slda_coefficients[k]])
    
indices = np.array(slda_coefficients).argsort()
data = np.array(data)
data = data[indices]

pd.DataFrame(data, columns=["Topic", "Suicidality Coefficient"])

,Topic,Suicidality Coefficient
0,"person, big, babes, girl, sexy, ’, f, teen, bo...",-5.75108528137207
1,"mail, order, person, seeds, biodynamic, organi...",-5.470564365386963
2,"url, us, min, survey, gt95, person, short, stu...",-5.056283950805664
3,"person, vs, persons, 10, team, 1, 2, 3, round,...",-4.8567986488342285
4,"person, porn, real, app, iphone, tube, mobile,...",-4.844600677490234
5,"live, webcam, sexy, ’, sex, rough, blood, ince...",-4.4133195877075195
6,"person, amazon, video, build, power, atx, back...",-4.2340898513793945
7,"person, fit, shirt, size, wearing, wear, bra, ...",-4.027881622314453
8,"person, music, video, song, url, love, favorit...",-3.900761127471924
9,"gt, xb1, ps4, psn, person, vog, x1, invite, lv...",-3.820251226425171


In [146]:
print(data[39][0])

im, dont, know, like, want, feel, get, ive, really, cant, people, life, even, going, much, time, tell, things, need, never, think, love, way, right, still, help, friends, go, someone, one, didnt, thing, since, sure, hate, na, doesnt, ill, anything, everything


In [ ]:
vector = slda.get_topic_vecs(model, filtered_data)

  0%|          | 3/29200 [00:00<17:23, 27.98it/s]

Getting topic distributions...


 99%|█████████▊| 28829/29200 [14:37<00:11, 32.84it/s]

In [ ]:
#Uncomment to print example feature vector
#print(vector['hw4uh'][0])

In [ ]:
X = np.array([ vector[key][0] for key in vector.keys()])
y = np.array([ vector[key][1] for key in vector.keys()])
y= y.reshape(np.shape(y)[0])

In [ ]:
print(np.shape(X))
print(np.shape(y))

In [ ]:
p_clf = post_classifier.PostClassification("RbfSVM")
p_clf.train(X,y)

In [ ]:
y_pred = p_clf.test(X)

In [ ]:
p_clf.get_metrics(y, y_pred)

In [ ]:
print(sum(y_pred))
print(sum(y))